In [5]:
# Bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from datetime import timedelta
import os
from pathlib import Path
#from lightgbm import LGBMRegressor
from mlforecast import MLForecast
from sklearn.ensemble import HistGradientBoostingRegressor, RandomForestRegressor
from window_ops.rolling import rolling_mean
from mlforecast.lag_transforms import ExpandingMean, RollingMean
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import joblib
from tqdm import tqdm

In [6]:
#definir o diretório
os.chdir(os.getcwd()[:-9])

In [7]:
#importar dataset
df = pd.read_parquet("data/processed/vendas_engineered.parquet")

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147414 entries, 0 to 147413
Data columns (total 29 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   cfop           147414 non-null  int64         
 1   nnf            147414 non-null  int64         
 2   codfil         147414 non-null  int64         
 3   xnomeemit      147414 non-null  object        
 4   xfantemit      147414 non-null  object        
 5   cnpjdest       147414 non-null  int64         
 6   xmun           147414 non-null  object        
 7   cod            147414 non-null  float64       
 8   seqprod        147414 non-null  int64         
 9   xprod          147414 non-null  object        
 10  tipo           147414 non-null  object        
 11  qcom           147414 non-null  float64       
 12  ucom           147414 non-null  object        
 13  ano            147414 non-null  int64         
 14  mes            147414 non-null  int64         
 15  

In [20]:
# Ordenar por produto e data
df = df.sort_values(["cluster", "data"])

# Separar treino/teste
dias_teste = 6
train = (
    df.groupby("cluster", group_keys=False)
      .apply(lambda x: x.iloc[:-7])
      .reset_index(drop=True)
)

test = (
    df.groupby("cluster", group_keys=False)
      .apply(lambda x: x.iloc[-7:])
      .reset_index(drop=True)
)

<positron-console-cell-20>:8: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
<positron-console-cell-20>:14: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


In [27]:
# Treinar modelo com MLForecast e avaliação com erro médio ponderado
def erro_ponderado_por_qtd(df_aval, col_pred):
    erros = []
    for uid, grupo in df_aval.groupby("cluster"):
        if len(grupo) < 1:
            continue
        erro = root_mean_squared_error(grupo["qcom"], grupo[col_pred])
        peso = len(grupo)
        erros.append((erro, peso))

    total_pesos = sum(p for _, p in erros)
    if total_pesos == 0:
        return np.nan

    erro_ponderado = sum(e * p for e, p in erros) / total_pesos
    return erro_ponderado

def rodar_mlforecast(df_train, df_test):
    models = [
        #LGBMRegressor(
        #    random_state=42,
        #    verbosity=-1,
        #    min_child_samples=30,  # Maior valor para evitar overfitting
        #    reg_alpha=0.2,        # Regularização L1
        #    reg_lambda=0.2,       # Regularização L2
        #    n_estimators=100
        #),
        HistGradientBoostingRegressor(
            random_state=42,
            min_samples_leaf=30,
            l2_regularization=0.2,
            max_iter=100
        ),
        RandomForestRegressor(
            random_state=42,
            n_estimators = 100
        )
    ]

    fcst = MLForecast(
        models=models,
        freq='M',
        lags=[1, 7, 14],
        lag_transforms={
            1: [ExpandingMean()],
            3: [RollingMean(window_size=3)],
            7: [RollingMean(window_size=7)],
            14: [RollingMean(window_size=14)]
        },
        date_features=["mes", "ano"]
    )

    fcst.fit(df_train, 
             id_col="cluster", 
             time_col="data", 
             target_col="qcom", 
             dropna=False,
             weight_col="sample_weight",
             static_features=[]
             )
    pred = fcst.predict(7)

    df_aval = df_test[["cluster", "data", "qcom"]].merge(pred, on=["cluster", "data"], how="inner")
    resultados = []
    best_mape = 1
    for col in pred.columns:
        if col.startswith("LGBM") or col.startswith("Hist") or col.startswith("Rand"):
            mae = mean_absolute_error(df_aval["qcom"], df_aval[col])
            mape = mean_absolute_percentage_error(df_aval["qcom"], df_aval[col])
            erro_pond = erro_ponderado_por_qtd(df_aval, col)
            resultados.append((col, mae, mape, erro_pond))
            print(f"{col}:\n MAE: {mae:.2f}\n MAPE: {mape:.2%}\n Erro Ponderado: {erro_pond:.2f}\n")

            if mape < best_mape:
                modelo = col
                best_mape = mape

    best_model = {modelo: best_mape}

    return pred, best_model

In [ ]:
# Executar
predicoes, best_model = rodar_mlforecast(train, test)

In [ ]:
# Treinar modelo com MLForecast e avaliação com erro médio ponderado
def erro_ponderado_por_qtd(df_aval, col_pred):
    erros = []
    for uid, grupo in df_aval.groupby("unique_id"):
        if len(grupo) < 1:
            continue
        erro = root_mean_squared_error(grupo["y"], grupo[col_pred])
        peso = len(grupo)
        erros.append((erro, peso))

    total_pesos = sum(p for _, p in erros)
    if total_pesos == 0:
        return np.nan

    erro_ponderado = sum(e * p for e, p in erros) / total_pesos
    return erro_ponderado

def is_weekend(dates):
    """É final de semana"""
    return dates.dayofweek.isin([5, 6]).astype(int)

def day_month(dates):
    """dia do mês"""
    return dates.day

def month(dates):
    """mes do ano"""
    return dates.month

def rodar_mlforecast(df_train, df_test, save_path='melhores_modelos'):
    # Criar diretório para salvar os modelos se não existir
    Path(save_path).mkdir(parents=True, exist_ok=True)
    
    models = [
        ('LGBM', LGBMRegressor(
            random_state=42,
            verbosity=-1,
            min_child_samples=30,
            reg_alpha=0.2,
            reg_lambda=0.2,
            n_estimators=100
        )),
        ('HGBR', HistGradientBoostingRegressor(
            random_state=42,
            min_samples_leaf=30,
            l2_regularization=0.2,
            max_iter=100
        )),
        ('RF', RandomForestRegressor(
            random_state=42,
            n_estimators=100
        ))
    ]

    # Criar dicionário para mapear nomes de colunas para modelos
    model_map = {
        'LGBMRegressor': models[0][1],
        'HistGradientBoostingRegressor': models[1][1],
        'RandomForestRegressor': models[2][1]
    }

    fcst = MLForecast(
        models=[m[1] for m in models],
        freq='D',
        lags=[1, 3, 7],
        lag_transforms={
            1: [ExpandingMean()],
            3: [RollingMean(window_size=3)],
            7: [RollingMean(window_size=7)],
            14: [RollingMean(window_size=14)]
        },
        date_features=["dayofweek", "dayofyear", is_weekend, day_month, month]
    )

    fcst.fit(df_train, 
             id_col="unique_id", 
             time_col="ds", 
             target_col="y", 
             dropna=False,
             weight_col="sample_weight",
             static_features=["product_cluster"]
             )
    
    # Fazer previsões
    pred = fcst.predict(7)

    # Avaliar modelos
    df_aval = df_test[["unique_id", "ds", "y"]].merge(pred, on=["unique_id", "ds"], how="inner")
    resultados = []
    best_metrics = {
        'model_name': None,
        'model': None,
        'mae': float('inf'),
        'mape': float('inf'),
        'erro_ponderado': float('inf')
    }

    for col in pred.columns:
        if any(col.startswith(m[0]) for m in models):
            mae = mean_absolute_error(df_aval["y"], df_aval[col])
            mape = mean_absolute_percentage_error(df_aval["y"], df_aval[col])
            erro_pond = erro_ponderado_por_qtd(df_aval, col)
            
            resultados.append({
                'model_name': col,
                'mae': mae,
                'mape': mape,
                'erro_ponderado': erro_pond
            })
            
            print(f"{col}:\n MAE: {mae:.2f}\n MAPE: {mape:.2%}\n Erro Ponderado: {erro_pond:.2f}\n")

            # Atualizar melhor modelo se encontrar um MAPE menor
            if mape < best_metrics['mape']:
                best_metrics.update({
                    'model_name': col,
                    'model': model_map[col],
                    'mae': mae,
                    'mape': mape,
                    'erro_ponderado': erro_pond
                })

    # Salvar o melhor modelo
    if best_metrics['model_name']:
        model_info = {
            'model': best_metrics['model'],
            'model_name': best_metrics['model_name'],
            'metrics': {
                'mae': best_metrics['mae'],
                'mape': best_metrics['mape'],
                'erro_ponderado': best_metrics['erro_ponderado']
            },
            'mlforecast_config': {
                'freq': 'D',
                'lags': [1, 3, 7],
                'lag_transforms': {
                    1: [ExpandingMean()],
                    3: [RollingMean(window_size=3)],
                    7: [RollingMean(window_size=7)],
                    14: [RollingMean(window_size=14)]
                },
                'date_features': ["dayofweek", "dayofyear", is_weekend, day_month, month]
            },
            'fit_params': {
                'id_col': "unique_id",
                'time_col': "ds", 
                'target_col': "y",
                'static_features': ["product_cluster"]
            }
        }
        
        # Salvar o objeto do modelo
        model_path = f"{save_path}/{best_metrics['model_name']}_best_model.pkl"
        joblib.dump(model_info, model_path)
        print(f"\nMelhor modelo salvo em: {model_path}")
        print(f"Modelo: {best_metrics['model_name']} com MAPE: {best_metrics['mape']:.2%}")

    return pred, resultados, best_metrics

In [260]:
# Treinar e avaliar modelos
previsoes, resultados, melhor_modelo = rodar_mlforecast(train, test)

c:\Users\marce\anaconda3\Lib\site-packages\mlforecast\core.py:625: UserWarning: Found null values in lag3, lag7, expanding_mean_lag1, rolling_mean_lag3_window_size3, rolling_mean_lag7_window_size7, rolling_mean_lag14_window_size14.
  warnings.warn(f'Found null values in {", ".join(cols_with_nulls)}.')
c:\Users\marce\anaconda3\Lib\site-packages\mlforecast\core.py:625: UserWarning: Found null values in lag3, lag7, expanding_mean_lag1, rolling_mean_lag3_window_size3, rolling_mean_lag7_window_size7, rolling_mean_lag14_window_size14.
  warnings.warn(f'Found null values in {", ".join(cols_with_nulls)}.')
c:\Users\marce\anaconda3\Lib\site-packages\mlforecast\core.py:625: UserWarning: Found null values in lag7, expanding_mean_lag1, rolling_mean_lag3_window_size3, rolling_mean_lag7_window_size7, rolling_mean_lag14_window_size14.
  warnings.warn(f'Found null values in {", ".join(cols_with_nulls)}.')
c:\Users\marce\anaconda3\Lib\site-packages\mlforecast\core.py:625: UserWarning: Found null values

LGBMRegressor:
 MAE: 6.29
 MAPE: 27.16%
 Erro Ponderado: 7.66


Melhor modelo salvo em: melhores_modelos/LGBMRegressor_best_model.pkl
Modelo: LGBMRegressor com MAPE: 27.16%


In [ ]:
# ----------------------------
# Funções auxiliares
# ----------------------------
def erro_ponderado_por_qtd(df_aval, col_pred):
    erros = []
    for uid, grupo in df_aval.groupby("unique_id"):
        if len(grupo) < 1:
            continue
        erro = root_mean_squared_error(grupo["y"], grupo[col_pred])
        peso = len(grupo)
        erros.append((erro, peso))

    total_pesos = sum(p for _, p in erros)
    if total_pesos == 0:
        return np.nan

    erro_ponderado = sum(e * p for e, p in erros) / total_pesos
    return erro_ponderado

def is_weekend(dates):
    return pd.Series(dates).dt.dayofweek.isin([5, 6]).astype(int)

def day_month(dates):
    return pd.Series(dates).dt.day

def month(dates):
    return pd.Series(dates).dt.month


# ----------------------------
# Estratégia para produtos
# ----------------------------

def split_por_volume(df, limiar_curto=12, limiar_medio=30):
    contagens = df['unique_id'].value_counts()
    ids_curto = contagens[contagens < limiar_curto].index
    ids_medio = contagens[(contagens >= limiar_curto) & (contagens < limiar_medio)].index
    ids_longo = contagens[contagens >= limiar_medio].index

    return ids_curto, ids_medio, ids_longo


# ----------------------------
# Média móvel simples para séries curtas
# ----------------------------

def media_movel_simples(df, janela=3, dias_prever=7):
    resultados = []

    for uid, grupo in df.groupby("unique_id"):
        grupo = grupo.sort_values("ds")
        media = grupo["y"].tail(janela).mean()
        datas_futuras = pd.date_range(grupo["ds"].max() + pd.Timedelta(days=1), periods=dias_prever)
        preds = pd.DataFrame({
            "unique_id": uid,
            "ds": datas_futuras,
            "y_pred": media
        })
        resultados.append(preds)

    return pd.concat(resultados, ignore_index=True)


# ----------------------------
# Forecast com MLForecast
# ----------------------------

def rodar_mlforecast(df_train, df_test):
    models = [
        LGBMRegressor(
            random_state=42,
            verbosity=-1,
            min_child_samples=30,
            reg_alpha=0.2,
            reg_lambda=0.2,
            n_estimators=100
        ),
        HistGradientBoostingRegressor(
            random_state=42,
            min_samples_leaf=30,
            l2_regularization=0.2,
            max_iter=100
        ),
        RandomForestRegressor(
            random_state=42,
            n_estimators=100
        )
    ]

    fcst = MLForecast(
        models=models,
        freq='D',
        lags=[1, 3, 7, 14],
        lag_transforms={
            1: [ExpandingMean()],
            3: [RollingMean(window_size=3)],
            7: [RollingMean(window_size=7)],
            14: [RollingMean(window_size=14)]
        },
        date_features=["dayofweek", "dayofyear", is_weekend, day_month, month]
    )

    fcst.fit(
        df_train,
        id_col="unique_id",
        time_col="ds",
        target_col="y",
        dropna=False,
        weight_col="sample_weight" if "sample_weight" in df_train.columns else None,
        static_features=["product_cluster"] if "product_cluster" in df_train.columns else None
    )

    pred = fcst.predict(7)

    df_aval = df_test[["unique_id", "ds", "y"]].merge(pred, on=["unique_id", "ds"], how="inner")
    resultados = []
    best_mape = 1
    modelo = None

    for col in pred.columns:
        if col.startswith(("LGBM", "Hist", "Rand")):
            mae = mean_absolute_error(df_aval["y"], df_aval[col])
            mape = mean_absolute_percentage_error(df_aval["y"], df_aval[col])
            erro_pond = erro_ponderado_por_qtd(df_aval, col)
            resultados.append((col, mae, mape, erro_pond))
            print(f"{col}:\n MAE: {mae:.2f} | MAPE: {mape:.2%} | Erro Ponderado: {erro_pond:.2f}")

            if mape < best_mape:
                modelo = col
                best_mape = mape

    best_model = {modelo: best_mape}
    return pred, best_model


# ----------------------------
# Estratégia com divisão por volume de dados
# ----------------------------

def pipeline_previsao(df, dias_prever=7):
    ids_curto, ids_medio, ids_longo = split_por_volume(df)

    df_curto = df[df['unique_id'].isin(ids_curto)]
    df_medio_longo = df[df['unique_id'].isin(ids_medio.union(ids_longo))]

    print(f"🔹 Produtos com poucos dados: {len(ids_curto)}")
    print(f"🔹 Produtos com dados suficientes: {len(ids_medio.union(ids_longo))}")

    # Previsão por média móvel
    pred_curto = media_movel_simples(df_curto, dias_prever=dias_prever)

    # Previsão por MLForecast para os demais
    df_train = df_medio_longo.groupby("unique_id", group_keys=False).apply(
        lambda g: g.sort_values("ds").iloc[:-dias_prever] if len(g) > dias_prever else g
    ).reset_index(drop=True)
    df_test = df_medio_longo[~df_medio_longo.index.isin(df_train.index)]

    pred_ml, best_model = rodar_mlforecast(df_train, df_test)
    pred_ml.rename(columns={list(best_model.keys())[0]: "y_pred"}, inplace=True)

    # Combina as previsões
    previsoes = pd.concat([pred_curto, pred_ml[["unique_id", "ds", "y_pred"]]], ignore_index=True)

    # Calcular erro global
    df_real = df[df["ds"].isin(previsoes["ds"].unique())]
    df_merge = df_real.merge(previsoes, on=["unique_id", "ds"], how="inner")
    erro_segmentado = calcular_erro_total(df_merge)

    return previsoes.sort_values(["unique_id", "ds"]), erro_segmentado


# ----------------------------
# Estratégia: aplicar MLForecast para todos
# ----------------------------

def previsao_todos_mlforecast(df, dias_prever=7):
    df = df.sort_values(["unique_id", "ds"])
    
    df_train = df.groupby("unique_id", group_keys=False).apply(
        lambda g: g.iloc[:-dias_prever] if len(g) > dias_prever else g
    ).reset_index(drop=True)

    df_test = df[~df.index.isin(df_train.index)]

    pred, best_model = rodar_mlforecast(df_train, df_test)
    pred.rename(columns={list(best_model.keys())[0]: "y_pred"}, inplace=True)

    df_merge = df_test.merge(pred[["unique_id", "ds", "y_pred"]], on=["unique_id", "ds"], how="inner")
    erro_total = calcular_erro_total(df_merge)

    return df_merge.sort_values(["unique_id", "ds"]), erro_total


# ----------------------------
# Execução e Comparação Final
# ----------------------------

def comparar_estrategias(df):
    print("\n🔍 Rodando estratégia segmentada por volume de histórico...")
    previsoes_segmentado, erro_segmentado = pipeline_previsao(df)
    print(f"\n📊 Erro ponderado - Estratégia segmentada: {erro_segmentado:.2f}")

    print("\n🔍 Rodando estratégia com MLForecast para todos...")
    previsoes_ml, erro_ml = previsao_todos_mlforecast(df)
    print(f"📊 Erro ponderado - MLForecast para todos: {erro_ml:.2f}")

    print("\n📈 Resultado final:")
    if erro_segmentado < erro_ml:
        print("✅ Melhor usar abordagem segmentada por volume de histórico.")
    else:
        print("✅ Melhor aplicar MLForecast para todos os produtos.")

    return previsoes_segmentado, previsoes_ml

In [13]:
previsoes_segmentado, previsoes_ml = comparar_estrategias(df)


🔍 Rodando estratégia segmentada por volume de histórico...
🔹 Produtos com poucos dados: 2822
🔹 Produtos com dados suficientes: 2874


C:\Users\marce\AppData\Local\Temp\ipykernel_22032\547070314.py:163: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_train = df_medio_longo.groupby("unique_id", group_keys=False).apply(
c:\Users\marce\anaconda3\Lib\site-packages\mlforecast\core.py:625: UserWarning: Found null values in lag7, lag14, rolling_mean_lag7_window_size7, rolling_mean_lag14_window_size14.
  warnings.warn(f'Found null values in {", ".join(cols_with_nulls)}.')
c:\Users\marce\anaconda3\Lib\site-packages\mlforecast\core.py:625: UserWarning: Found null values in lag7, lag14, rolling_mean_lag7_window_size7, rolling_mean_lag14_window_size14.
  warnings.warn(f'Found null values in {", ".join(cols_with_nulls)}.')
c:\Users\marce\an

LGBMRegressor:
 MAE: 6.19 | MAPE: 27.09% | Erro Ponderado: 7.47
HistGradientBoostingRegressor:
 MAE: 6.23 | MAPE: 29.35% | Erro Ponderado: 7.45
RandomForestRegressor:
 MAE: 6.11 | MAPE: 28.26% | Erro Ponderado: 7.35

📊 Erro ponderado - Estratégia segmentada: 7.84

🔍 Rodando estratégia com MLForecast para todos...


C:\Users\marce\AppData\Local\Temp\ipykernel_22032\547070314.py:189: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_train = df.groupby("unique_id", group_keys=False).apply(
c:\Users\marce\anaconda3\Lib\site-packages\mlforecast\core.py:625: UserWarning: Found null values in lag3, lag7, lag14, expanding_mean_lag1, rolling_mean_lag3_window_size3, rolling_mean_lag7_window_size7, rolling_mean_lag14_window_size14.
  warnings.warn(f'Found null values in {", ".join(cols_with_nulls)}.')
c:\Users\marce\anaconda3\Lib\site-packages\mlforecast\core.py:625: UserWarning: Found null values in lag3, lag7, lag14, expanding_mean_lag1, rolling_mean_lag3_window_size3, rolling_mean_lag7_window_size7, rolling_mean_lag

LGBMRegressor:
 MAE: 5.50 | MAPE: 26.49% | Erro Ponderado: 6.60
HistGradientBoostingRegressor:
 MAE: 5.52 | MAPE: 27.48% | Erro Ponderado: 6.56
RandomForestRegressor:
 MAE: 5.83 | MAPE: 28.48% | Erro Ponderado: 6.98
📊 Erro ponderado - MLForecast para todos: 6.60

📈 Resultado final:
✅ Melhor aplicar MLForecast para todos os produtos.
